In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim

In [16]:
# Завантаження даних
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
dataset = pd.read_csv(url, header=None)

In [17]:
# Визначення ознак та міток
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

# Поділ даних на тренувальну і валідаційну вибірку (70/30%)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

# Нормалізація даних
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_val_normalized = scaler.transform(X_val)


In [18]:
# Визначення архітектури мережі
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 12)
        self.fc2 = nn.Linear(12, 8)
        self.fc3 = nn.Linear(8, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

# Ініціалізація мережі та функції втрат
input_size = X_train_normalized.shape[1]
net = Net(input_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [19]:
# Тренування мережі
epochs = 300
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = net(torch.tensor(X_train_normalized, dtype=torch.float32))
    loss = criterion(outputs, torch.tensor(y_train, dtype=torch.float32).view(-1, 1))
    loss.backward()
    optimizer.step()

# Оцінка точності на валідаційній вибірці
with torch.no_grad():
    outputs = net(torch.tensor(X_val_normalized, dtype=torch.float32))
    predicted = (outputs > 0.5).float()
    accuracy = (predicted == torch.tensor(y_val, dtype=torch.float32).view(-1, 1)).float().mean()
    print(f'Точність на валідаційній вибірці: {accuracy.item():.2f}')

# Побудова кореляційної матриці
correlation_matrix = dataset.corr()

# Вибір перших 4 ознак з кореляційної матриці
selected_features = correlation_matrix.index[:4]
X_train_selected = X_train_normalized[:, selected_features]
X_val_selected = X_val_normalized[:, selected_features]

# Перевизначення та тренування мережі з новими даними
input_size_selected = X_train_selected.shape[1]
net_selected = Net(input_size_selected)
optimizer_selected = optim.Adam(net_selected.parameters(), lr=0.001)

# Тренування мережі для підмножини ознак
for epoch in range(epochs):
    optimizer_selected.zero_grad()
    outputs = net_selected(torch.tensor(X_train_selected, dtype=torch.float32))
    loss = criterion(outputs, torch.tensor(y_train, dtype=torch.float32).view(-1, 1))
    loss.backward()
    optimizer_selected.step()



Точність на валідаційній вибірці: 0.71
